In [2]:
from __future__ import print_function, division
import SimpleITK as sitk
import numpy as np
import os
from glob import glob
import pandas as pd
from tqdm import tqdm
import numpy as np
from load_tools import get_cube_from_img, load_itk_version2

In [ ]:
num_subsets = 10
image_size = 32

for subsetindex in tqdm(range(num_subsets)):
    luna_path = "/datagrid/Medical/nodules/Luna/"
    luna_subset_path = luna_path + "subset" + str(subsetindex) + "/"
    output_path = "/datagrid/temporary/dobkomar/output_path_32/"
    file_list = glob(luna_subset_path + "*.mhd")

    luna_csv_path = "/datagrid/Medical/nodules/Luna/"
    df_node = pd.read_csv('/home.stud/dobkomar/candidates_V2.csv')
    df_node["file"] = df_node["seriesuid"].map(lambda file_name: [f for f in file_list if file_name in f][0])
    df_node = df_node.dropna()
    
    
    for fcount, img_file in enumerate(tqdm(file_list)):
        temp_df = df_node[df_node["file"] == img_file]

        if temp_df.shape[0] > 0:
            img_array, origin, spacing = load_itk_version2(img_file)

            index = 0
            for node_idx, row in temp_df.iterrows():
                x, y, z = row["coordX"], row["coordY"], row["coordZ"]
                label = row["class"]

                center = np.array([x, y, z])
                v_center = np.rint((center - origin) / spacing)
                
                # convert x,y,z order  to z,y,x  
                v_center[0], v_center[1], v_center[2] = v_center[2], v_center[1], v_center[0]
                
                node_cube = get_cube_from_img(img_array, v_center, image_size)
                node_cube.astype(np.uint8)

                if label == 1:
                    filepath = output_path + "1/"
                    if not os.path.exists(filepath):
                        os.makedirs(filepath)
                    filename = str(subsetindex) + "_" + str(fcount) + "_" + str(index) + '_' + str(row['seriesuid']) + '_' + str(x) + '_' + str(y) + '_' + str(z) +'_'+ str(label)  
                    np.save(filepath + filename + ".npy", node_cube)

                if label == 0:
                    filepath = output_path + "0/"
                    if not os.path.exists(filepath):
                        os.makedirs(filepath)
                    filename = str(subsetindex) + "_" + str(fcount) + "_" + str(index) + '_' + str(row['seriesuid']) + '_' + str(x) + '_' + str(y) + '_' + str(z) +'_'+ str(label)   
                    np.save(filepath + filename + ".npy", node_cube)
                index += 1
